
# Algoritmo K-means: Versión Serial y Paralela

## Introducción

El algoritmo **K-means** es uno de los métodos más populares y ampliamente usados en tareas de agrupamiento no supervisado (*clustering*). Su objetivo principal es organizar un conjunto de datos en grupos o *clusters*, donde cada observación pertenece al grupo cuyo centro (centroide) es el más cercano según una medida de distancia, comúnmente la distancia euclidiana.

En este notebook, presentaremos dos implementaciones del algoritmo K-means:

- **Versión Serial**: Una implementación sencilla del algoritmo sin optimizaciones especiales, ejecutada secuencialmente.
- **Versión Paralela**: Una versión optimizada utilizando OpenMP para aprovechar múltiples núcleos del CPU y reducir significativamente el tiempo de ejecución.

Ambas versiones han sido implementadas en C++.

---

## Descripción del Problema

Utilizaremos un archivo CSV que contiene una gran cantidad de puntos en dos dimensiones (X, Y). Nuestro objetivo es:

- Asignar cada punto a uno de \( k \) grupos.
- Determinar la posición de cada centroide de modo que la distancia total entre los puntos y sus centroides sea mínima.

Cada fila del archivo CSV tiene el siguiente formato:

```
X,Y
```

---

## Estructuras y Funciones Básicas (Comunes a ambas versiones)

Comencemos describiendo detalladamente las estructuras y funciones auxiliares comunes a ambas versiones (serial y paralela):

### Estructura `Point`

Esta estructura representa un punto en el espacio bidimensional:

```cpp
struct Point {
    double x;       // Coordenada X
    double y;       // Coordenada Y
    int cluster;    // Índice del cluster asignado al punto (inicialmente -1)

    Point(double x, double y) : x(x), y(y), cluster(-1) {}
};
```

- `x` y `y` almacenan la ubicación del punto en el plano 2D.
- `cluster` almacena el índice del centroide más cercano. Inicialmente, este valor es `-1`, indicando que el punto aún no está asignado.

---

### Estructura `Centroid`

Esta estructura representa un centroide, que actúa como centro de cada grupo de puntos:

```cpp
struct Centroid {
    double x;                 // Coordenada X del centroide
    double y;                 // Coordenada Y del centroide
    std::vector<int> points;  // Índices de los puntos asignados al centroide

    Centroid(double x, double y) : x(x), y(y) {}

    void clear() {
        points.clear();       // Limpia la asignación de puntos antes de cada iteración
    }
};
```

- `x` y `y` representan la posición actual del centroide.
- `points` es una lista con los índices de los puntos actualmente asignados a este centroide. Esta lista se reinicia en cada iteración del algoritmo mediante el método `clear()`.

---

### Función para Calcular Distancia Euclidiana

La función `distance` calcula la distancia euclidiana entre un punto y un centroide específico:

```cpp
double distance(const Point &point, const Centroid &centroid) {
    return std::sqrt(std::pow(point.x - centroid.x, 2) + 
                     std::pow(point.y - centroid.y, 2));
}
```

- Esta función es fundamental para determinar qué tan cerca está un punto respecto a cada centroide.
- La fórmula usada es la distancia euclidiana estándar:


$$
\text{distancia}(p, c) = \sqrt{(p_x - c_x)^2 + (p_y - c_y)^2}
$$

---

## Estrategia General del Algoritmo K-means (Serial)

La versión serial del algoritmo K-means sigue estos pasos generales:

1. **Inicializar centroides**:
   - Se seleccionan aleatoriamente \( k \) posiciones dentro del rango de los datos como centroides iniciales.

2. **Asignar puntos**:
   - Cada punto se asigna al centroide más cercano según la distancia euclidiana.

3. **Actualizar centroides**:
   - Se recalcula la posición de cada centroide como el promedio de todos los puntos asignados a él.

4. **Repetir pasos 2 y 3** hasta que:
   - Ningún punto cambie de grupo (convergencia), o se alcance el número máximo de iteraciones.

---

## Código Serial: Carga de Datos

El método `loadPoints` carga puntos desde un archivo CSV:

```cpp
bool loadPoints(const std::string &filename) {
    std::ifstream file(filename);
    if (!file.is_open()) {
        std::cerr << "Error: No se pudo abrir el archivo " << filename << std::endl;
        return false;
    }

    points.clear();
    std::string line;

    while (std::getline(file, line)) {
        std::stringstream ss(line);
        std::string x_str, y_str;

        if (std::getline(ss, x_str, ',') && std::getline(ss, y_str, ',')) {
            try {
                double x = std::stod(x_str);
                double y = std::stod(y_str);
                points.emplace_back(x, y);
            }
            catch (const std::exception &e) {
                std::cerr << "Error al convertir a número: " << e.what() << std::endl;
            }
        }
    }

    file.close();

    if (points.empty()) {
        std::cerr << "Error: No se cargaron puntos desde el archivo." << std::endl;
        return false;
    }

    return true;
}
```

- Este método lee línea por línea, separa por comas, convierte a números reales, y los almacena en un vector de `Point`.

## Guardar Resultados  

El método `saveResults` guarda los resultados en un archivo CSV:

```cpp
bool saveResults(const std::string &filename)
    {
        std::ofstream file(filename);
        if (!file.is_open())
        {
            std::cerr << "Error: No se pudo abrir el archivo " << filename << " para escritura." << std::endl;
            return false;
        }

        // Escribir cada punto con su cluster asignado
        for (const auto &point : points)
        {
            file << std::fixed << std::setprecision(3)
                 << point.x << ","
                 << point.y << ","
                 << point.cluster << std::endl;
        }

        file.close();
        return true;
    }

- Este método genera un archivo CSV con tres columnas (`x, y, cluster`) indicando la posición y asignación final de cada punto.


## Algoritmo K-means Serial: Detalle Paso a Paso del Código

### Paso 1: Inicialización Aleatoria de Centroides

El método `initializeCentroids` elige aleatoriamente \( k \) posiciones dentro del rango de valores de los puntos cargados:

```cpp
void initializeCentroids() {
    if (points.empty()) {
        std::cerr << "Error: No hay puntos para inicializar centroides." << std::endl;
        return;
    }

    centroids.clear();

    // Encontrar los mínimos y máximos en cada dimensión
    double min_x = std::numeric_limits<double>::max();
    double max_x = std::numeric_limits<double>::min();
    double min_y = std::numeric_limits<double>::max();
    double max_y = std::numeric_limits<double>::min();

    for (const auto &point : points) {
        min_x = std::min(min_x, point.x);
        max_x = std::max(max_x, point.x);
        min_y = std::min(min_y, point.y);
        max_y = std::max(max_y, point.y);
    }

    // Distribuciones uniformes para coordenadas aleatorias
    std::uniform_real_distribution<double> dist_x(min_x, max_x);
    std::uniform_real_distribution<double> dist_y(min_y, max_y);

    // Crear k centroides con posiciones aleatorias
    for (int i = 0; i < k; i++) {
        double x = dist_x(rng);
        double y = dist_y(rng);
        centroids.emplace_back(x, y);
    }
}
```

**Explicación:**

- Este método primero determina los valores mínimos y máximos en X y Y de los puntos cargados.
- Luego genera centroides en posiciones aleatorias, asegurando que estén distribuidos dentro del rango de los datos.

---

### Paso 2: Asignación de Puntos al Centroide más Cercano

El método `assignPointsToCentroids` asigna cada punto al centroide más cercano según la distancia euclidiana:

```cpp
bool assignPointsToCentroids() {
    bool changed = false;

    // Limpiar las asignaciones anteriores
    for (auto &centroid : centroids) {
        centroid.clear();
    }

    // Asignar cada punto al centroide más cercano
    for (size_t i = 0; i < points.size(); i++) {
        double min_distance = std::numeric_limits<double>::max();
        int closest_centroid = -1;

        // Calcular distancia a cada centroide
        for (size_t j = 0; j < centroids.size(); j++) {
            double dist = distance(points[i], centroids[j]);
            if (dist < min_distance) {
                min_distance = dist;
                closest_centroid = j;
            }
        }

        // Verificar y actualizar asignación del cluster
        if (closest_centroid != -1) {
            if (points[i].cluster != closest_centroid) {
                changed = true; // Indica que hubo un cambio de asignación
                points[i].cluster = closest_centroid;
            }
            centroids[closest_centroid].points.push_back(i);
        }
    }

    return changed;
}
```

**Explicación:**

- Para cada punto, se determina cuál es el centroide más cercano según la distancia euclidiana.
- Si la asignación del punto cambia respecto a la iteración anterior, `changed` se vuelve verdadero.
- Cada centroide guarda los índices de los puntos que le pertenecen.

---

### Paso 3: Actualización de las posiciones de los centroides

El método `updateCentroids` actualiza la posición de cada centroide usando la media de todos los puntos asignados a él:

```cpp
void updateCentroids() {
    for (size_t i = 0; i < centroids.size(); i++) {
        if (centroids[i].points.empty()) {
            continue; // Si no tiene puntos asignados, no actualizar posición
        }

        double sum_x = 0.0;
        double sum_y = 0.0;

        // Calcular el promedio de las posiciones de los puntos asignados
        for (size_t j = 0; j < centroids[i].points.size(); j++) {
            int point_idx = centroids[i].points[j];
            sum_x += points[point_idx].x;
            sum_y += points[point_idx].y;
        }

        // Actualizar la posición del centroide
        centroids[i].x = sum_x / centroids[i].points.size();
        centroids[i].y = sum_y / centroids[i].points.size();
    }
}
```

**Explicación:**

- Para cada centroide, se suman las coordenadas de los puntos asignados.
- La nueva posición del centroide es el promedio de estas coordenadas.
- Esto asegura que el centroide se mueva hacia el centro del grupo de puntos.

---

### Ejecución del Algoritmo

Finalmente, la función `run` controla el flujo general del algoritmo:

```cpp
void run(int max_iterations = 100) {
    if (points.empty() || k <= 0) {
        std::cerr << "Error: No hay puntos o el número de clusters es inválido." << std::endl;
        return;
    }

    initializeCentroids();

    bool changed = true;
    int iteration = 0;

    // Iterar hasta convergencia o máximo de iteraciones
    while (changed && iteration < max_iterations) {
        changed = assignPointsToCentroids();
        updateCentroids();
        iteration++;
    }
}
```

**Explicación:**

- Se inicializan los centroides una sola vez al comienzo.
- Luego, se repiten los pasos de asignación y actualización hasta que ningún punto cambie de cluster o se alcance un máximo número de iteraciones definido.
